# Police Shooting Data With Census and Crime Data Covariates

This notebook will create two combined datasets and three individual datasets. The goal is to combine as much information at the county level to accompany police shooting databases as to facilitate predictions and data visualizations of police shooting data.

The datasets to be created are as follows:

1. **[Census data with various county-level features (2014)](#census_df)**
2. **[Police Shooting data from 2013-2016](#shooting_df)**
3. **[FBI county-level crime data (2014)](#crime_df)**
4. **[Combined county-level data with census, police shooting, and crime data](#full_df)**
5. **[Police Shooting data with county-level crime and census covariates](#shooting_county_df)**

Where possible intermediate datasets are downloaded directly from source. Otherwise they are stored in the ``datasets`` directory. Furthermore, all derived dataframes will be stored in ``derived_datasets``.

In [1]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from urllib2 import Request, urlopen
import json
import os
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon
from shapely.prepared import prep

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<a id='census_df'></a>

# 1. Create Census Dataframe

Here will use county level census data and shapefiles

* **Population demographics**
    * County Characteristics Datasets: Annual County Resident Population Estimates by Age, Sex, Race, and Hispanic Origin: April 1, 2010 to July 1, 2015
    * https://www2.census.gov/programs-surveys/popest/datasets/2010-2015/counties/asrh/cc-est2015-alldata.csv
    
* **County level shapefiles**
    * Cartographic Boundary Shapefiles - Counties
    * http://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_us_county_5m.zip
    
* **Income inequality index (GINI)**    
    * 2010-2014 American Community Survey 5-Year Estimates GINI index
    * http://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_14_5YR_B19083&prodType=table

* **Income and Poverty Estimates**
    * 2014 Small Area Income and Poverty Estimates, county and state estimates
    * https://www.census.gov/did/www/saipe/downloads/estmod14/est14ALL.xls
    
* **Educational attainment**
    * 2010-2014 American Community Survey 5-Year Estimate
    * http://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_15_5YR_S1501&src=pt
* **Unemployment Rate**
    * 2014 Local Area Unemployment Statistics (BLS)
    *  https://www.bls.gov/lau/laucnty14.xlsx

## Get Census data and make base dataframe

In [2]:
#cdf = pd.read_csv('https://www.census.gov/popest/data/counties/asrh/2015/files/CC-EST2015-ALLDATA.csv')
cdf = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2015/counties/asrh/cc-est2015-alldata.csv')
cdf.to_csv('datasets/CC-EST2015-ALLDATA.csv')

# select out 2014 data and all ages. We use 2014 since that 
# is the year of the crime statistics that we will use later
cdf = cdf[np.logical_and(cdf['AGEGRP']==0, cdf['YEAR']==7)]

# make new column of total FIPS number
cdf['FIPS'] = map(lambda x, y: '%02d'%x+'%03d'%y, cdf['STATE'], cdf['COUNTY'])

# sort by FIPS and reset index
cdf.sort_values('FIPS', inplace=True)
cdf.reset_index(inplace=True)

In [3]:
# hack to get county level shapefiles. I'm sure there is a better python way to do this...
cmd = 'mkdir -p datasets/shapefiles/us_county; wget http://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_us_county_5m.zip -O datasets/shapefiles/us_county/temp.zip; cd datasets/shapefiles/us_county; unzip temp.zip; rm temp.zip; cd ../../../'
os.system(cmd);

## Read Census County data into geopandas dataframe and merge

In [4]:
ccnty = gpd.read_file('datasets/shapefiles/us_county/cb_2015_us_county_5m.shp')

# define combined fips code
ccnty['FIPS'] = ccnty.apply(lambda x: '{0:02d}{1:03d}'.format(int(x['STATEFP']), int(x['COUNTYFP'])), axis=1)

# only keep US states + PR
statefp = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', 
           '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', 
           '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', 
           '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', 
           '54', '55', '56']

ccnty = ccnty[map(lambda x: x in statefp, ccnty['STATEFP'])]

# sort by fips code
ccnty.sort_values('FIPS', inplace=True)
ccnty.reset_index(inplace=True)

In [5]:
# add in population data
keys = [u'FIPS', u'STNAME', u'TOT_POP', u'TOT_MALE', u'TOT_FEMALE', u'WA_MALE',
       u'WA_FEMALE', u'BAC_MALE', u'BAC_FEMALE', u'AAC_MALE', 
        u'AAC_FEMALE', u'H_MALE', u'H_FEMALE']
ccnty = ccnty.merge(cdf[keys], on='FIPS', how='outer')

## Get Race percentages by county

In [6]:
keys = ['WA', 'BAC', 'AAC', 'H']
for key in keys:
    ccnty[key+'_PER'] = (ccnty[key+'_MALE'] + ccnty[key+'_FEMALE']) / ccnty['TOT_POP']

## Get GINI data, clean, and merge

In [7]:
gini = pd.read_csv('datasets/ACS_14_5YR_B19083_with_ann.csv')
gini['FIPS'] = gini.apply(lambda x: '{0:05d}'.format(int(x['FIPS'])), axis=1)
gini.rename(columns={'Estimate; Gini Index': 'GINI'}, inplace=True)

# merge
ccnty = pd.merge(ccnty, gini[['FIPS', 'GINI']], on='FIPS')

## Get income and poverty data, clean, and merge

In [8]:
# read in excel (why don't they have csv!!!???)
incm = pd.read_excel('https://www.census.gov/did/www/saipe/downloads/estmod14/est14ALL.xls', header=3)

# output csv for conviencienc
incm.to_csv('datasets/est14ALL.csv')

In [9]:
# get combined state + county fips
incm['FIPS'] = incm.apply(lambda x: '{0:02d}{1:03d}'.format(int(x['State FIPS Code']), 
                                                            int(x['County FIPS Code'])), axis=1)

# replace names
keydict = {'Median Household Income': 'median_household_income', 'Poverty Percent, All Ages': 'percent_in_poverty'}
incm.rename(columns=keydict, inplace=True)

# get rid of blank rows and convert to mean values
incm['median_household_income'] = incm['median_household_income'].apply(str)
incm['percent_in_poverty'] = incm['percent_in_poverty'].apply(str)
incm.replace(to_replace={'median_household_income': {'.': 0}, 'percent_in_poverty': {'.':0}}, inplace=True)
incm['median_household_income'] = incm['median_household_income'].apply(int)
incm['percent_in_poverty'] = incm['percent_in_poverty'].apply(float)
incm.replace(to_replace={'median_household_income': {0: incm['median_household_income'].mean()}, 
                         'percent_in_poverty': {0:incm['percent_in_poverty'].mean()}}, inplace=True)


In [10]:
# merge
ccnty = pd.merge(ccnty, incm[['FIPS', 'median_household_income', 'percent_in_poverty']], on='FIPS', how='left')

## Get education attainment data, clean and merge

In [11]:
ed = pd.read_csv('datasets/ACS_14_5YR_S1501_with_ann.csv')
ed['FIPS'] = ed.apply(lambda x: '{0:05d}'.format(int(x['FIPS'])), axis=1)

# just pull out total numbers of high school or higher and bachelor and higher
keydict = {'Total; Estimate; Percent high school graduate or higher': 'edu_total_hs', 
           'Male; Estimate; Percent high school graduate or higher': 'edu_male_hs',
           'Female; Estimate; Percent high school graduate or higher': 'edu_female_hs',
           'Total; Estimate; Percent bachelor\'s degree or higher': 'edu_total_bach',
           'Male; Estimate; Percent bachelor\'s degree or higher': 'edu_male_bach',
           'Female; Estimate; Percent bachelor\'s degree or higher': 'edu_female_bach'}
ed.rename(columns=keydict, inplace=True)

# merge
keys = ['FIPS', 'edu_total_hs', 'edu_male_hs', 'edu_female_hs', 
        'edu_total_bach', 'edu_male_bach', 'edu_female_bach']
ccnty = pd.merge(ccnty, ed[keys], on='FIPS', how='left')

## Get unemployment data and merge

In [12]:
# explicitly specify column names
names = ['laus','stfips','countyfips','cntyname','year', 'blank',
         'laborfc','employed','unemployed','unemployment_rate']
emp = pd.read_excel('https://www.bls.gov/lau/laucnty14.xlsx', header=6, names=names)

# drop NaNs
emp.dropna(inplace=True, subset=['stfips','countyfips', 'unemployment_rate'])

# get FIPS
emp['FIPS'] = map(lambda x, y: '%02d'%int(x)+'%03d'%int(y), emp['stfips'], emp['countyfips'])

keys = ['FIPS', 'unemployment_rate']
ccnty = pd.merge(ccnty, emp[keys], on='FIPS', how='left')

## Save in csv format

In [13]:
ccnty.to_csv('derived_datasets/county_level_census_data.csv', encoding='utf-8', 
             index=False, columns=ccnty.columns[1:])

<a id='shooting_df'></a>

# 2. Create police shooting dataframe

Here we create a dataset from 2013 - 2016 looking at police shooting incidents. We make use of two different datasets for 2013-2014 and 2015-2016

* **2015-2016** [Washington Post Police Shooting Database](https://www.washingtonpost.com/graphics/national/police-shootings-2016/)
    * 2015: https://s3.amazonaws.com/postgraphics/policeshootings/policeshootings2015.json
    * 2016: https://s3.amazonaws.com/postgraphics/policeshootings/policeshootings2016.json
* **2013-2014** [Mapping Police Violence Database](http://mappingpoliceviolence.org)
    * http://mappingpoliceviolence.org/s/MPVDatasetDownload-4kjm.xlsx

## Get Washington Post data and merge

In [14]:
# get 2015 and 2016 data from Washington Post
df15 = pd.read_json('https://s3.amazonaws.com/postgraphics/policeshootings/policeshootings2015.json')
df16 = pd.read_json('https://s3.amazonaws.com/postgraphics/policeshootings/policeshootings2016.json')

In [15]:
# keep only relevant columns
keys = ['name', 'age', 'blurb', 'description', 'date', 'gender', 'race', 'lon', 'lat', 'state', 
        'armed', 'mental', 'weapon', 'flee', 'is_body_camera']
frames = [df15[keys], df16[keys]]
dftot = pd.concat(frames, ignore_index=True)

# make sure all lowercase
keys = ['armed', 'weapon', 'flee', 'blurb', 'description']
dftot[keys] = dftot[keys].apply(lambda x: x.str.lower())

# maked armed boolean column
dftot['armed_bool'] = map(lambda x: x not in ['unarmed'] and x not in ['no weapon'], dftot['armed'].values)

# add in year column
dftot['year'] = dftot['date'].apply(lambda x: x.year)

## Use county boundaries to determine FIPS for each shooting

This is quite expensive and there is probably a better and more efficient way but this seems to work...

In [16]:
# define set of Points
pts = pd.Series([Point(x, y) for x, y in zip(dftot['lon'].values, dftot['lat'].values)])
s_points = MultiPoint(list(pts.values))

# define Polygon for each FIPS number
fips_polygon = prep(MultiPolygon(list(ccnty['geometry'].values)))

# determine points and indices that are within boundaries
#fips_points = filter(fips_polygon.contains, pts)
fips_points, fips_ind = [], []
for ct, fp in enumerate(pts):
    if np.any(ccnty['geometry'].map(lambda x: prep(x).contains(fp))):
        fips_points.append(fp)
        fips_ind.append(True)
    else:
        fips_ind.append(False)
#fips_ind = map(fips_polygon.contains, pts)

# get FIPS number corresponding to each shooting
fips = map(lambda y: ccnty['FIPS'][ccnty['geometry'].map(lambda x: prep(x).contains(y))].values[0], fips_points)

# remap dftot
dftot = dftot[fips_ind]
dftot['FIPS'] = fips

## Read, clean, and filter MPV data

In [17]:
mpv = pd.read_excel('http://mappingpoliceviolence.org/s/MPVDatasetDownload-4kjm.xlsx', encoding='utf-8')

In [18]:
### do some cleaning to make responses same as WP data

# rename some keys to match WP data
keydict = {'Victim\'s name': 'name', 'Victim\'s age': 'age', 
           'Victim\'s gender': 'gender', 'Victim\'s race': 'race', 
           'Date of injury resulting in death (month/day/year)': 'date', 
           'A brief description of the circumstances surrounding the death': 'description', 
           'Symptoms of mental illness?': 'mental', 'Unarmed': 'armed', 
           'Location of death (state)': 'state'}
mpv.rename(columns=keydict, inplace=True)

# lowercase and filter out
mpv['mental'] = mpv['mental'].apply(lambda x: str(x).lower().strip())
mpv['mental'] = mpv['mental'].apply(lambda x: x not in ['no', 'unknown', 'nan'])

# fill NaN for unknown age
mpv.replace(to_replace={'age': {'Unknown':np.nan, 'nan':np.nan}}, inplace=True)

# lower description
mpv['description'] = mpv['description'].apply(lambda x: unicode(x).lower())

# convert dates to datetime objects
mpv['date'] = pd.to_datetime(mpv['date'])
mpv['year'] = mpv['date'].apply(lambda x: x.year)

# convert genders
mpv.replace(to_replace={'gender': {'Unknown':np.nan, 'Male':'M', 'Female':'F', 
                                   'Transgender':np.nan}}, inplace=True)

# convert races
mpv.replace(to_replace={'race': {'Unknown race':'N', 'White':'W', 'Black':'B', 
                                 'Hispanic':'H', 'Asian':'A', 'Native American':'O', 
                                 'Pacific Islander':'O'}}, inplace=True)

# get armed status (should probably create a status for unknown)
mpv['armed_bool'] = mpv['armed'] != 'Unarmed'

# get cause of death, try to separte out shootings since that is all WP data has
mpv['shot'] = mpv['Cause of death'].apply(lambda y: 'gunshot' in [x.lower().strip() for x in y.split(',')])

# short hand for zip code
mpv['zip'] = mpv['Location of death (zip code)']

# now filter out all 2015 and 2016 deaths (assume they are in WP data) ...could cross check
ind = np.logical_or(mpv['year'].values==2016, mpv['year'].values==2015)
mpvf = mpv[~ind]

# also filter out all non-gunshot deaths
mpvf = mpvf[mpvf['shot'].values==True]

# now filter out deaths without zip codes (could do geocoding later but this is ok for counties)
mpvf = mpvf[mpvf['zip'].notnull()]

# convert zip to str
mpvf['zip'] = mpvf['zip'].map(lambda x: '%05d'%int(x))

In [19]:
# get zip2fip data and make dictionary
z2f = pd.read_excel('https://www.huduser.gov/portal/datasets/usps/ZIP_COUNTY_092016.xlsx')

z2f['ZIP'] = map(lambda x: '%05d'%(int(x)), z2f['ZIP'].values)
z2f['FIPS'] = map(lambda x: '%05d'%(int(x)), z2f['COUNTY'].values)

zips, fips = z2f['ZIP'].values, z2f['FIPS'].values
zip2fip = {}
for z, f in zip(zips, fips):
    zip2fip[z] = f 

In [20]:
# get fips code
keys = zip2fip.keys()
mpvf['FIPS'] = map(lambda x: zip2fip[x] if x in keys else np.nan, mpvf['zip'].values)

## Combine with WP data and save

In [21]:
# keep on certain keys
keys = ['FIPS', 'name', 'age', 'description', 'date', 'year', 'gender', 'race', 'state', 
        'armed_bool', 'mental']
mpvf = mpvf[keys]

dftot = pd.concat([dftot, mpvf], join='outer', axis=0)
dftot = dftot[keys]

In [22]:
dftot.to_csv('derived_datasets/combined_wp_mpv_shooting.csv', encoding='utf-8', 
             index=False)

<a id='crime_df'></a>
# 3. Create FBI Crime dataframe

* Uniform Crime Reporting Program Data: Arrests by Age, Sex, and Race, 2014 (ICPSR 36394)
    * http://www.icpsr.umich.edu/icpsrweb/NACJD/series/57/studies/36394
* Law Enforcement Agency Indentifiers Crosswalk, 2012 (ICPSR 35158)
    * http://www.icpsr.umich.edu/icpsrweb/NACJD/studies/35158

## Read in crime data in tsv format

In [23]:
cd = pd.read_table('datasets/36394-0001-Data.tsv', na_values=['99999', '99998'])

/Users/jaellis/Envs/ps/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Get FIPS value from FBI Crosswalk file

In [24]:
# read in file
cross = pd.read_table('datasets/35158-0001-Data.tsv')
cross['FIPS'] = cross.apply(lambda x: '{0:05d}'.format(int(x['FIPS'])), axis=1)

# make ORI >- FIPS dict
ori_fips_dict = {}
oris = cross['ORI7'].values
fips = cross['FIPS'].values
for ori, fip in zip(oris, fips):
    ori_fips_dict[ori] = fip

In [25]:
# get FIPS code
keys = ori_fips_dict.keys()
cd['FIPS'] = map(lambda x: ori_fips_dict[x] if x in keys else np.nan, cd['ORI'].values)

# filter out data that has fips codes
cd = cd[cd['FIPS'].notnull()]

## Create crime categories

Create broad crime categories for violent, property, drugs, and weapon crimes. 

Offense codes from datasets/36394-0001-Codebook.pdf 

In [26]:
# violent crime (homicide, aggravated assault, forcible rape, robbery)
keys = ['01A', '02', '03', '04']
ind = cd['OFFENSE'].map(lambda x: x.strip() in keys)
cd.ix[ind, 'crime'] = 'violent'

# property crime (arson, burglary, larceny-theft, motor vehicle theft)
keys = ['05', '06', '07', '09']
ind = cd['OFFENSE'].map(lambda x: x.strip() in keys)
cd.ix[ind, 'crime'] = 'property'

# drug charges
keys = ['18', '180', '185', '18A', '18B', '18C', '18D', '18E', 
        '18F', '18G', '18H']
ind = cd['OFFENSE'].map(lambda x: x.strip() in keys)
cd.ix[ind, 'crime'] = 'drug'

# weapons chages
keys = ['15']
ind = cd['OFFENSE'].map(lambda x: x.strip() in keys)
cd.ix[ind, 'crime'] = 'weapon'

In [27]:
keys = ['FIPS', 'crime', 'AW', 'AB', 'AA', 'JW', 'JB', 'JA', 'AH', 'JH']
cd = cd[keys]

In [28]:
cd.to_csv('derived_datasets/county_crime_data.csv', encoding='utf-8', 
             index=False)

## Get arrest counts by race, crime, FIPS code

In [29]:
# groupby fips and crime and sum arrests
cd_sum = cd.groupby(['FIPS', 'crime']).agg(np.nansum)

In [30]:
# get total arrests counts for white, black, asian, hispanic
races = ['W', 'B', 'A', 'H']
races2 = ['white', 'black', 'asian', 'hispanic']
for race, race2 in zip(races, races2):
    cd_sum['arrest_count_{0}'.format(race2)] = cd_sum['A{0}'.format(race)] + cd_sum['J{0}'.format(race)]
    
# filter out columns again
keys = ['AW', 'AB', 'AA', 'JW', 'JB', 'JA', 'AH', 'JH']
cd_sum.drop(labels=keys, inplace=True, axis=1)

# reset index
cd_sum.reset_index(inplace=True)

In [ ]:
cd_sum.to_csv('derived_datasets/summed_county_crime_data.csv', encoding='utf-8', 
             index=False)

<a id='full_df'></a>

# 4. Make full county dataframe with census data, shooting data and crime data

## Add crime data to census data 

Use census population data to determine the crime rate for the crime categories determined above.

**Note:** In some cases, the crime rate is extremely large because of low populations in some counties. Furthermore, I assume the arrest count can have repeat offenders.

In [ ]:
crimes = ['property', 'drug', 'violent', 'weapon']
races = ['white', 'black', 'hispanic', 'asian']
racecodes = ['WA', 'BAC', 'H', 'AAC']


## Probably a more clever and more efficient way to do this... ##
for crime in crimes:
    ccnty['crime_rate_tot_{0}'.format(crime)] = 0.0
    for race in races:
        ccnty['crime_rate_{0}_{1}'.format(race, crime)] = 0.0
        
for fips in ccnty['FIPS']:
    ind = ccnty['FIPS'] == fips
    for crime in crimes:
        
        # filter out fips code
        xf = cd_sum[cd_sum['FIPS'] == fips]
        
        # filter out crime code
        xf = xf[xf['crime']==crime]
        
        # check to see if any data
        if len(xf) > 0:
            keys = ['arrest_count_black', 'arrest_count_white', 'arrest_count_hispanic']
            arrests = xf[keys].sum(axis=1).values[0]
            pop = ccnty[ind]['TOT_POP'].values[0]
            if arrests >= 1:
                ccnty.ix[ind, 'crime_rate_tot_{0}'.format(crime)] =  arrests / pop * 1000
            
            for race, rc in zip(races, racecodes):
                arrests = xf['arrest_count_{0}'.format(race)].values[0]
                pop = (ccnty[ind]['{0}_MALE'.format(rc)]+ccnty[ind]['{0}_FEMALE'.format(rc)]).values[0]
                #if arrests/pop > 0.1:
                    #print(ccnty[ind]['NAME'].values[0], ccnty[ind]['STNAME'].values[0], fips, arrests, pop, race, 
                    #       crime, ccnty[ind]['BAC_PER'].values[0])
                if arrests >= 1:
                    ccnty.ix[ind, 'crime_rate_{0}_{1}'.format(race, crime)] = arrests / pop * 1000

## Add Shooting data counts to combined data frame

Add both counts and counts/yr to get an average shooting rate since 2013

In [ ]:
# get total shootings
counts = dftot['FIPS'].value_counts()

# avg shooting over years
tdiff = (dftot['date'].max() - dftot['date'].min()).days / 365.25
counties = pd.DataFrame({'FIPS': counts.index, 'shootings': counts, 'shooting_avg':counts/tdiff})

# merge in shooting data
ccnty = ccnty.merge(counties, on='FIPS', how='outer')

# filter out NaN and replace with 0
ccnty.replace(to_replace={'shootings': {np.nan: 0}, 'shooting_avg': {np.nan: 0}}, inplace=True)

# get shooting rate per 1,000,000 people
ccnty['shooting_rate'] = ccnty['shooting_avg'] / ccnty['TOT_POP'] * 1000000

In [ ]:
# get shooting by race by armed status
races = [('W', 'WA', 'white'), ('B', 'BAC', 'black'), ('H', 'H', 'hispanic'), ('A', 'AAC', 'asian')]
armed = ['armed', 'unarmed']
for rc, rk, rv in races:
    ind = dftot['race']==rc
    counts = dftot[ind]['FIPS'].value_counts()
    counties = pd.DataFrame({'FIPS': counts.index, 'shootings_{0}_total'.format(rv): counts, 
                             'shooting_avg_{0}_total'.format(rv): counts/tdiff})
    ccnty = ccnty.merge(counties, on='FIPS', how='outer')
    ccnty.replace(to_replace={'shootings_{0}_total'.format(rv): {np.nan: 0}, 
                              'shooting_avg_{0}_total'.format(rv): {np.nan: 0}}, inplace=True)
    pop = ccnty['{0}_MALE'.format(rk)] + ccnty['{0}_FEMALE'.format(rk)]
    ccnty['shootings_{0}_total_rate'.format(rv)] = ccnty['shooting_avg_{0}_total'.format(rv)] / pop * 1000000
    for arm in armed:
        ind = np.logical_and(dftot['race']==rc, dftot['armed_bool']==(arm=='armed'))
        counts = dftot[ind]['FIPS'].value_counts()
        counties = pd.DataFrame({'FIPS': counts.index, 'shootings_{0}_{1}'.format(arm, rv): counts, 
                                 'shooting_avg_{0}_{1}'.format(arm, rv): counts/tdiff})
        ccnty = ccnty.merge(counties, on='FIPS', how='outer')
        ccnty.replace(to_replace={'shootings_{0}_{1}'.format(arm, rv): {np.nan: 0}, 
                                  'shooting_avg_{0}_{1}'.format(arm, rv): {np.nan: 0}}, inplace=True)
        pop = ccnty['{0}_MALE'.format(rk)] + ccnty['{0}_FEMALE'.format(rk)]
        ccnty['shootings_{0}_{1}_rate'.format(arm, rv)] = ccnty['shooting_avg_{0}_{1}'.format(arm, rv)] / pop * 1000000

In [ ]:
ccnty.to_csv('derived_datasets/full_combined_county_data.csv', encoding='utf-8', 
             index=False, columns=ccnty.columns[1:])

<a id='shooting_county_df'></a>

# 5. Add County level data to police shooting dataset 

This way each shooting will now be accompanied by the corresponding county demographics, census info, and crime statistics.

In [ ]:
keys = list(ccnty.columns[24:57])
keys.append('FIPS')

df = pd.merge(dftot, ccnty[keys], on='FIPS')

In [ ]:
df.to_csv('derived_datasets/shooting_data_with_county_covariates.csv', encoding='utf-8', 
             index=False)